In [30]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [31]:
# 1.How would you read this dataset into pandas dataframe?
df = pd.read_csv(r"C:\Users\bhanu\Downloads\Healtcare Data.csv")

In [32]:
# 2. How can you check the missing or null values in the dataset?
df.isna().sum() # There are no null or missing values

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

In [33]:
# 3.how can you View the first 10 rows and the structure(datatypes and columns) of the DataFrame

print(df.head(10))
print(df.dtypes)
print(df.columns)


      PatientId  AppointmentID Gender          ScheduledDay  \
0  2.987250e+13        5642903      F  2016-04-29T18:38:08Z   
1  5.589978e+14        5642503      M  2016-04-29T16:08:27Z   
2  4.262962e+12        5642549      F  2016-04-29T16:19:04Z   
3  8.679512e+11        5642828      F  2016-04-29T17:29:31Z   
4  8.841186e+12        5642494      F  2016-04-29T16:07:23Z   
5  9.598513e+13        5626772      F  2016-04-27T08:36:51Z   
6  7.336882e+14        5630279      F  2016-04-27T15:05:12Z   
7  3.449833e+12        5630575      F  2016-04-27T15:39:58Z   
8  5.639473e+13        5638447      F  2016-04-29T08:02:16Z   
9  7.812456e+13        5629123      F  2016-04-27T12:48:25Z   

         AppointmentDay  Age      Neighbourhood  Scholarship  Hipertension  \
0  2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0             1   
1  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             0   
2  2016-04-29T00:00:00Z   62      MATA DA PRAIA            0            

In [34]:
# 4.How would you get the number of unique patients in the dataset?
no_of_unique_patients = df['PatientId'].nunique()
print(no_of_unique_patients)

62299


In [35]:
# 5.how would you check for duplicate rows based on patient ID and appointmentID?
duplicate_rows = df[df.duplicated(subset=['PatientId','AppointmentID'])]
duplicate_rows

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show


In [36]:
# 6.How do you calculate the percentage of patients who did not show up for their appointment?
notAttended = df[df['No-show'] == 'Yes']
no_show_rate = (len(notAttended)/len(df))*100
print(f"no show rate {no_show_rate:.2f}")

no show rate 20.19


In [37]:
# 7.How can you group the data by gender and count the no-shows in each group? 
no_show_counts = df[df['No-show']=='Yes'].groupby('Gender')['No-show'].count()
no_show_counts

Gender
F    14594
M     7725
Name: No-show, dtype: int64

In [38]:
# 8.How would you find the average age of patients who showed up versus those who didn't? 
showed = df[df['No-show'] == 'No']
not_showed = df[df['No-show'] == 'Yes']

print(f'avg of age of showed patients = {showed['Age'].mean():.2f}')
print(f'avg of age of not_showed patients = {not_showed['Age'].mean():.2f}')


avg of age of showed patients = 37.79
avg of age of not_showed patients = 34.32


In [39]:
# 9. how can you  count how many patients received SMS reminders but still didn't show up. 
sms_received = df[df['SMS_received'] == 1]
no_show_after_sms = sms_received[sms_received['No-show'] == 'Yes']
print(f"No of patients didnt show up after receiving sms remainder = {no_show_after_sms.shape[0]}")

No of patients didnt show up after receiving sms remainder = 9784


In [40]:
# 10.How would you determine the neighborhood with the highest no-show rate? 
total_appointments = df.groupby('Neighbourhood').size()
no_shows = df[df['No-show'] == 'Yes'].groupby('Neighbourhood').size()
no_show_rate = (no_shows / total_appointments) * 100
highest_neighbourhood = no_show_rate.idxmax()
highest_rate = no_show_rate.max()

print(f"Neighborhood with highest no-show rate: {highest_neighbourhood}")
print(f"No-show rate: {highest_rate:.2f}%")


Neighborhood with highest no-show rate: ILHAS OCEÂNICAS DE TRINDADE
No-show rate: 100.00%


In [41]:
# 11.How would you convert ScheduleDay and Appointment Day to Datetime Object? 
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])
df.dtypes



PatientId                     float64
AppointmentID                   int64
Gender                         object
ScheduledDay      datetime64[ns, UTC]
AppointmentDay    datetime64[ns, UTC]
Age                             int64
Neighbourhood                  object
Scholarship                     int64
Hipertension                    int64
Diabetes                        int64
Alcoholism                      int64
Handcap                         int64
SMS_received                    int64
No-show                        object
dtype: object

In [42]:
# 12. How do you calculate the number of days between Schedule Date and Appointment Date? 
df['DaysBetween'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days
df['DaysBetween'] = df['DaysBetween'].apply(lambda x : 0 if x<=-1 else x)
df['DaysBetween']

0          0
1          0
2          0
3          0
4          0
          ..
110522    34
110523    34
110524    40
110525    40
110526    40
Name: DaysBetween, Length: 110527, dtype: int64

In [43]:
# 13. How would you find the Average Scheduling Lead Time for No Shows vs Shows? 
df['LeadTime'] = (df['AppointmentDay'].dt.date - df['ScheduledDay'].dt.date).apply(lambda x: x.days)
avg_lead_time = df.groupby('No-show')['LeadTime'].mean()

print("Average Scheduling Lead Time (in days):")
print(avg_lead_time)

Average Scheduling Lead Time (in days):
No-show
No      8.754659
Yes    15.831489
Name: LeadTime, dtype: float64


In [44]:
# 14. How can you filter appointments that were scheduled on the same day as the appointment?

df[df['ScheduledDay'].dt.date == df['AppointmentDay'].dt.date]


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,DaysBetween,LeadTime
0,2.987250e+13,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0,0
1,5.589978e+14,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,No,0,0
2,4.262962e+12,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,No,0,0
3,8.679512e+11,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,0,0
4,8.841186e+12,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110511,8.235996e+11,5786742,F,2016-06-08 08:50:20+00:00,2016-06-08 00:00:00+00:00,14,MARIA ORTIZ,0,0,0,0,0,0,No,0,0
110512,9.876246e+13,5786368,F,2016-06-08 08:20:01+00:00,2016-06-08 00:00:00+00:00,41,MARIA ORTIZ,0,0,0,0,0,0,No,0,0
110513,8.674778e+13,5785964,M,2016-06-08 07:52:55+00:00,2016-06-08 00:00:00+00:00,2,ANTÔNIO HONÓRIO,0,0,0,0,0,0,No,0,0
110514,2.695685e+12,5786567,F,2016-06-08 08:35:31+00:00,2016-06-08 00:00:00+00:00,58,MARIA ORTIZ,0,0,0,0,0,0,No,0,0


In [45]:
# 15.How do you handle incorrect or outliered values in the Age column?
df = df[df['Age'] >= 0]
Q1 = df['Age'].quantile(0.25)
Q3 = df['Age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_cleaned = df[(df['Age'] >= lower_bound) & (df['Age'] <= upper_bound)]

In [46]:
# 16. how would you convert column name into snake case naming convention

def to_snake_case(name):
    name = name.replace('-', '_')
    name = re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()
    return name

df.columns = [to_snake_case(col) for col in df.columns]    
df.columns

Index(['patient_id', 'appointment_i_d', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hipertension', 'diabetes', 'alcoholism', 'handcap', 's_m_s_received',
       'no_show', 'days_between', 'lead_time'],
      dtype='object')

In [55]:

# 17. Convert the no show column into boolean
df['no_show'] = df['no_show'].map({'Yes':True,'No':False})

In [61]:
# 18. how you drop coulmns that are not relevant to the analysis?abs

df.drop(columns=['appointment_i_d'],inplace=True)
df


,patient_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,s_m_s_received,no_show,days_between,lead_time
0,2.987250e+13,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,NaN,0,0
1,5.589978e+14,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,NaN,0,0
2,4.262962e+12,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,NaN,0,0
3,8.679512e+11,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,NaN,0,0
4,8.841186e+12,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,F,2016-05-03 09:15:35+00:00,2016-06-07 00:00:00+00:00,56,MARIA ORTIZ,0,0,0,0,0,1,NaN,34,35
110523,3.596266e+12,F,2016-05-03 07:27:33+00:00,2016-06-07 00:00:00+00:00,51,MARIA ORTIZ,0,0,0,0,0,1,NaN,34,35
110524,1.557663e+13,F,2016-04-27 16:03:52+00:00,2016-06-07 00:00:00+00:00,21,MARIA ORTIZ,0,0,0,0,0,1,NaN,40,41
110525,9.213493e+13,F,2016-04-27 15:09:23+00:00,2016-06-07 00:00:00+00:00,38,MARIA ORTIZ,0,0,0,0,0,1,NaN,40,41


In [60]:
df

,patient_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,s_m_s_received,no_show,days_between,lead_time
0,2.987250e+13,F,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,NaN,0,0
1,5.589978e+14,M,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,NaN,0,0
2,4.262962e+12,F,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,NaN,0,0
3,8.679512e+11,F,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,NaN,0,0
4,8.841186e+12,F,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,F,2016-05-03 09:15:35+00:00,2016-06-07 00:00:00+00:00,56,MARIA ORTIZ,0,0,0,0,0,1,NaN,34,35
110523,3.596266e+12,F,2016-05-03 07:27:33+00:00,2016-06-07 00:00:00+00:00,51,MARIA ORTIZ,0,0,0,0,0,1,NaN,34,35
110524,1.557663e+13,F,2016-04-27 16:03:52+00:00,2016-06-07 00:00:00+00:00,21,MARIA ORTIZ,0,0,0,0,0,1,NaN,40,41
110525,9.213493e+13,F,2016-04-27 15:09:23+00:00,2016-06-07 00:00:00+00:00,38,MARIA ORTIZ,0,0,0,0,0,1,NaN,40,41


In [63]:
# 19. How would you prepare data to plot no-show rate by age group? 
df = pd.read_csv(r"C:\Users\bhanu\Downloads\Healtcare Data.csv")

age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = ['0-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100']

df['age_group'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=True)




C:\Users\bhanu\AppData\Local\Temp\ipykernel_14824\4098984430.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  no_show_rate = df.groupby('age_group')['No-show'].mean()


TypeError: agg function failed [how->mean,dtype->object]